In [146]:
# import libraries
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier, LinearSVC,OneVsRest,MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
#import featureExtractor.tfidf as tfidf
from functools import reduce
from nltk.corpus import stopwords

try:
    sc.stop()
except NameError:
    print(":)")
sc = SparkContext()
sqlContext = SQLContext(sc)

def readData(dataPath):
    #input: file path 
    #output: returns a spark dataframe 
    data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(dataPath)
    print("Data Read successfully ...")
    drop_list = ['url']
    
    data = data.select([column for column in data.columns if column not in drop_list])
    return data.dropna()



def tfidf(regexTokenizer, stopwordsRemover, label_stringIdx):
    #returns pipeline using tfidf 
    hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=5000)
    idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
    return Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

def countVectorizer(regexTokenizer, stopwordsRemover, label_stringIdx):
    # bag of words count
    countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
    label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
    return Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])   


def get_stopwords():
    stop_words = set(stopwords.words('english'))
    special = ['ms','mr','http','https','amp', 'none', 'i’m', 'th','don’t','it’s','advertisement']
    with open('seostopwords.txt') as f:
        stopLines = f.read().splitlines()

    dump = [stop_words.add(i) for i in special]
    dump = [stop_words.add(i) for i in stopLines]
    return list(stop_words)
    
def classify(featureExtractor, classifier):
    #input : 
        #1)feature extractor (count vectorizer or tfidf)
        #2) classifier algorithm (random forest, naive bayes or linear reg)
        
    # training :70% , testing 15% , validation 15%
    
    
    #Part 1:  Combine data from all sources 
    data_merged = readData('politics.csv')
    
    fnames = ['sports.csv','business.csv','movies.csv','tourism.csv','technology.csv',\
              'assault.csv','students and education.csv', 'terrorism.csv', 'med_and_health.csv',]
    
    for i in fnames:
        temp = readData(i)
        data_merged = data_merged.union(temp)
       
    data = data_merged.withColumnRenamed("category", "Category").withColumnRenamed("content", "Descript")
    print("The total number of articles from ", len(fnames)+1 ," categories is " , data.count())
        
    # Part 2 -> build regex tokenizer 
    regexTokenizer = RegexTokenizer(inputCol="Descript", outputCol="words", pattern="\\W")
    
    # stop words ---------- (3)
    add_stopwords = get_stopwords()
    #print(add_stopwords)
    
    # Part 3 -> build stop word tokenizer 
    stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered",stopWords=add_stopwords)
    #print(stopwordsRemover.transform())
    label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

    # Part 4 -> build pipeline 
    if featureExtractor=="cv":
        pipeline = countVectorizer(regexTokenizer, stopwordsRemover, label_stringIdx)
    elif featureExtractor=="tfidf":
        pipeline = tfidf(regexTokenizer, stopwordsRemover, label_stringIdx)
    print("Pipeline constructed successfully..")

    # Fit the pipeline to the model.
    pipelineFit = pipeline.fit(data)
    
    dataset = pipelineFit.transform(data)
    #dataset.show(1)
    dataset_row = str(dataset.select('features').take(1))
    num_feat = int(re.findall('(\d+?), \{',dataset_row)[0])
    
    # Part 5 -> split data and classify
    (trainingData, validationData, testData) = dataset.randomSplit([0.7, 0.15, 0.15], seed = 100)
    
    
    print("Training model..")
    if classifier=="lr":
        selectedModel = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
    elif classifier=="nb":
        selectedModel = NaiveBayes(smoothing=1)
    elif classifier=="rf":
        selectedModel = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 300, \
                            maxDepth = 20, \
                            maxBins = 2)
    elif classifier == 'perc':
        selectedModel = MultilayerPerceptronClassifier(maxIter=100, layers=[num_feat,156,75,10],\
                                                       blockSize=256, seed=1234)
    #elif classifier == 'svm':
    #    selectedModel = OneVsRest(classifier = LinearSVC(maxIter=10, regParam=0.1))

    model = selectedModel.fit(trainingData)
    print("Model Trained..")
    
    # part 6 -> predict and test model
    print("Test model..")
    predictions = model.transform(validationData)
    predictions.filter(predictions['prediction'] == 0) \
        .select("Descript","Category","probability","label","prediction") \
        .orderBy("probability", ascending=False)

    print("Evaluate model..")
    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
    accuracy = evaluator.evaluate(predictions)
    print("Accuracy: "+str(accuracy))

    

In [147]:
def main():
    print("Please select method for feature extraction. Type\ncv for count vectorizer\ntfidf for term \
frequency inverse document frequency")
    fe_choice = 'cv'
    fe_choice = input()
    print("\nPlese enter algorithm for classification. Type\nlr for linear regression\nnb for naive bayes\n\
rf for random forest")
    algo_choice = 'nb'
    algo_choice = input()
    classify(fe_choice,algo_choice)
    
    

In [148]:
main()

Please select method for feature extraction. Type
cv for count vectorizer
tfidf for term frequency inverse document frequency
cv

Plese enter algorithm for classification. Type
lr for linear regression
nb for naive bayes
rf for random forest
svm
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
Data Read successfully ...
The total number of articles from  10  categories is  467
Pipeline constructed successfully..
Training model..
Model Trained..
Test model..


AnalysisException: "cannot resolve '`probability`' given input columns: [words, label, prediction, Category, filtered, Descript, features];;\n'Project [Descript#33626, Category#33623, 'probability, label#33737, prediction#34758]\n+- AnalysisBarrier\n      +- Filter (prediction#34758 = cast(0 as double))\n         +- Project [Descript#33626, Category#33623, words#33722, filtered#33726, features#33731, label#33737, prediction#34758]\n            +- Project [Descript#33626, Category#33623, words#33722, filtered#33726, features#33731, label#33737, mbc$acc668beb02-be22-4f0c-a813-d1cfa7d9b716#34750, <lambda>(mbc$acc668beb02-be22-4f0c-a813-d1cfa7d9b716#34750) AS prediction#34758]\n               +- Project [Descript#33626, Category#33623, words#33722, filtered#33726, features#33731, label#33737, mbc$tmpf8e418f5-b5a0-4f06-a127-7fa80a3a6dd8#34733 AS mbc$acc668beb02-be22-4f0c-a813-d1cfa7d9b716#34750]\n                  +- Project [Descript#33626, Category#33623, words#33722, filtered#33726, feat

In [72]:
    cv    tfidf
lr 0.621  0.657
nb 0.822  0.782
rf 0.707  0.656
mp 0.796  0.786

SyntaxError: invalid syntax (<ipython-input-72-d1534a5b37fa>, line 1)